<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/getting_started/exploring_clinical_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with IDC clinical data without BigQuery

In this notebook we cover the basics of how you can access and search IDC clinical data without depending on Google BigQuery.

In addition to maintaining clinical data in Google BigQuery tables, we also export those in Parquet format into a public cloud-based storage bucket. Those files are free to download, and are rather small (as of IDC v18, less than 65MB altogether).

Once downloaded, you can search the content using Pandas sytax of SQL.

This brief notebook will guide you through the steps of the above.

---
Initial version: Jul 2024

## Prerequisites

The only prerequisite is [`idc-index`](https://github.com/ImagingDataCommons/idc-index) - python package that contains various utilities to simplify access to IDC data. As part of this package installation, you will get several other packages that we will use later:
* `s5cmd` for very efficient download of data from cloud buckets using S3 API
* `pandas` for dataframe operations
* `duckdb` for querying pandas dataframes using SQL syntax

In [ ]:
%%capture
!pip install --upgrade idc-index

## Download clinical data tables

In the following cell we will simply download all of the clinical data tables saved in Parquet format from a location in a cloud bucket.

In [ ]:
%%capture
!mkdir -p idc_clinical_data
!s5cmd --no-sign-request --endpoint-url https://s3.amazonaws.com cp 's3://idc-open-metadata/bigquery_export/idc_v18_clinical/*' ./idc_clinical_data

Once the cell above is completed, you should see the content of the `idc_clinical_data` directory that looks like the following:

```
/content/idc_clinical_data# tree .|more
.
├── acrin_6698_clinical
│   └── 000000000000.parquet
├── acrin_flt_breast_A0
│   └── 000000000000.parquet
├── acrin_flt_breast_A1
│   └── 000000000000.parquet
├── acrin_flt_breast_A3
│   └── 000000000000.parquet
...
```

You will see one folder for each table, where most of the folder names follow the pattern `<collection_id>_<table_name>` (i.e., in the example above `acrin_6698` is the name of the collection, and `clinical` is the name of a table provided by the submitters to accompany that collection, and `acrin_6698_clinical` is the table where that content was curated by IDC).

You will also see 2 special folders/tables:
* `table_metadata`: this is a table that lists all of the clinical data tables and provides table-level information
* `column_metadata`: think about this table as a dictionary containing metadata attributes (columns) across all of the clinical data tables

# Find all metadata available for the specific collection

A common use case is to find all clinical data available for a specific IDC collection. The key to answering this question is the `column_metadata` table.

In the following we will load this table into a pandas dataframe, and look at its columns.

In [ ]:
import duckdb
import pandas as pd

# load the table
column_metadata_df = pd.read_parquet('./idc_clinical_data/column_metadata')

column_metadata_df.columns


Index(['collection_id', 'case_col', 'table_name', 'column', 'column_label',
       'data_type', 'original_column_headers', 'values', 'values_source',
       'files', 'sheet_names', 'batch', 'column_numbers'],
      dtype='object')

The key columns of this dataframe are:
* `collection_id`: which collection given metadata attribute corresponds to
* `table_name`: the name of the table where this metadata attribute is located
* `column`: name of the column (attribute)

Depending on the specific attribute and how it was provided/documented by the submitter, you may find more information about it in the `column_label` column.

Let's assume we are interested in the clinical data accompanying the `rms_mutation_prediction` collection. We can select all clinical data attributes that are available for this collection as shown next.

In [ ]:
# define the query that selects all rows where collection_id is 'rms_mutation_prediction'
query = """
SELECT *
FROM column_metadata_df
WHERE collection_id = 'rms_mutation_prediction'
"""

# execute the query
matching_items = duckdb.sql(query).df()


In [ ]:
matching_items

,collection_id,case_col,table_name,column,column_label,data_type,original_column_headers,values,values_source,files,sheet_names,batch,column_numbers
0,rms_mutation_prediction,False,bigquery-public-data.idc_v18_clinical.rms_muta...,sample_id,sample_id,String,[['sample_id']],[],None,[CCDI_Submission_Template_v1.0.1_DM_v2.2023-02...,[sample],[0],[1]
1,rms_mutation_prediction,False,bigquery-public-data.idc_v18_clinical.rms_muta...,primary_site,primary_site,String,[['primary_site']],[],None,[CCDI_Submission_Template_v1.0.1_DM_v2.2023-02...,[diagnosis],[0],[5]
2,rms_mutation_prediction,True,bigquery-public-data.idc_v18_clinical.rms_muta...,participant_id,participant_id,String,[['participant_id']],[],None,[CCDI_Submission_Template_v1.0.1_DM_v2.2023-02...,[participant],[0],[0]
3,rms_mutation_prediction,False,bigquery-public-data.idc_v18_clinical.rms_muta...,age_at_diagnosis,age_at_diagnosis,float64,[['age_at_diagnosis']],[],None,[CCDI_Submission_Template_v1.0.1_DM_v2.2023-02...,[diagnosis],[0],[6]
4,rms_mutation_prediction,False,bigquery-public-data.idc_v18_clinical.rms_muta...,dicom_patient_id,idc_provenance_dicom_patient_id,String,[['idc_provenance_dicom_patient_id']],[],None,[],[],[],[]
5,rms_mutation_prediction,False,bigquery-public-data.idc_v18_clinical.rms_muta...,dicom_patient_id,idc_provenance_dicom_patient_id,String,[['idc_provenance_dicom_patient_id']],[],None,[],[],[],[]
6,rms_mutation_prediction,False,bigquery-public-data.idc_v18_clinical.rms_muta...,dicom_patient_id,idc_provenance_dicom_patient_id,String,[['idc_provenance_dicom_patient_id']],[],None,[],[],[],[]
7,rms_mutation_prediction,False,bigquery-public-data.idc_v18_clinical.rms_muta...,days_to_recurrence,days_to_recurrence,String,[['days_to_recurrence']],[],None,[CCDI_Submission_Template_v1.0.1_DM_v2.2023-02...,[diagnosis],[0],[9]
8,rms_mutation_prediction,False,bigquery-public-data.idc_v18_clinical.rms_muta...,sample_anatomic_site,sample_anatomic_site,String,[['sample_anatomic_site']],[],None,[CCDI_Submission_Template_v1.0.1_DM_v2.2023-02...,[sample],[0],[3]
9,rms_mutation_prediction,False,bigquery-public-data.idc_v18_clinical.rms_muta...,days_to_last_followup,days_to_last_followup,String,[['days_to_last_followup']],[],None,[CCDI_Submission_Template_v1.0.1_DM_v2.2023-02...,[diagnosis],[0],[13]


# Access table that contains specific metadata attribute

Let's assume we are interested in the `tumor_grade` attribute (row 13 in the table above). From the above, column `table_name` is telling us that it is contained in the table `bigquery-public-data.idc_v18_clinical.rms_mutation_prediction_sample`. The table name is the fully resolved name in BigQuery. For locating this table as downloaded, we need just the last component of the name: `rms_mutation_prediction_sample`.

We can access and search this table next.

In [ ]:
a_table_df = pd.read_parquet('./idc_clinical_data/rms_mutation_prediction_sample')

a_table_df

,dicom_patient_id,source_batch,participantparticipant_id,sample_id,sample_type,sample_anatomic_site,participant_age_at_collection,histological_classification,tumor_grade,tumor_stage_clinical_t,tumor_stage_clinical_n,tumor_stage_clinical_m,tumor_morphology,tumor_incidence_type,sample_description,sample_tumor_status
0,RMS2325,0,RMS2325,PAWDLM,Tumor,Leg,44.56,,,,,,,,,Tumor
1,RMS2124,0,RMS2124,PATMDI,Tumor,,0.90,BOTRYOID,,,,,,,,Tumor
2,RMS2137,0,RMS2137,PATVPL,Tumor,,0.83,BOTRYOID,,,,,,,,Tumor
3,RMS2140,0,RMS2140,PATYYW,Tumor,,1.07,BOTRYOID,,,,,,,,Tumor
4,RMS2145,0,RMS2145,PAUKHP,Tumor,,2.72,BOTRYOID,,,,,,,,Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,RMS2374,0,RMS2374,PAUPVA,Tumor,"Paratesticular, left",2.46,SPINDLE CELL RHABDOMYOSARCOMA,,,,,,,,Tumor
399,RMS2352,0,RMS2352,PASGZC,Tumor,"Paratesticular, right",0.68,SPINDLE CELL RHABDOMYOSARCOMA,,,,,,,,Tumor
400,RMS2205,0,RMS2205,PAMSJL,Tumor,Pelvis,2.76,MIXED ALVEOLAR AND EMBRYONAL RHABDOMYOSARCOMA,,,,,,,,Tumor
401,RMS2267,0,RMS2267,PALWAA,Tumor,"Soft tissue, abdomen",17.96,MIXED ALVEOLAR AND EMBRYONAL RHABDOMYOSARCOMA,,,,,,,,Tumor


Now that this table is loaded, you can search it as you would any pandas dataframe (or you can use SQL with duckdb, as shown earlier!).

Note that the `dicom_patient_id` column, which you will find in **every** clinical data table, can be used to link clinical metadata attributes to the DICOM image metadata!

## Summary

We hope you enjoyed this tutorial! If something didn't work as expected, if you have any feedback or suggestions of what should be added to this tutorial, please contact IDC support by sending email to support@canceridc.dev or posting your question on [IDC User forum](https://discourse.canceridc.dev).

## Acknowledgments

Imaging Data Commons has been funded in whole or in part with Federal funds from the National Cancer Institute, National Institutes of Health, under Task Order No. HHSN26110071 under Contract No. HHSN261201500003l.

If you use IDC in your research, please cite the following publication:

> Fedorov, A., Longabaugh, W. J. R., Pot, D., Clunie, D. A., Pieper, S. D., Gibbs, D. L., Bridge, C., Herrmann, M. D., Homeyer, A., Lewis, R., Aerts, H. J. W., Krishnaswamy, D., Thiriveedhi, V. K., Ciausu, C., Schacherer, D. P., Bontempi, D., Pihl, T., Wagner, U., Farahani, K., Kim, E. & Kikinis, R. _National Cancer Institute Imaging Data Commons: Toward Transparency, Reproducibility, and Scalability in Imaging Artificial Intelligence_. RadioGraphics (2023). https://doi.org/10.1148/rg.230180